# Finding the LRP signal with mne_pipeline

The `mne_pipeline` class contains a collection of helper and wrapper methods
for functions for the `mne` module.
Here we will take a look at how you would go about finding the
[lateralized readiness potential (LRP)](https://en.wikipedia.org/wiki/Lateralized_readiness_potential)
signal using the `mne_pipeline` class.

The data we will be using stems from an experiment where a participant had to choose
between two options of dividing money up between themselves and another anonymous
participant (i.e. a forced-choice version of the
[dictator game](https://en.wikipedia.org/wiki/Dictator_game)) by pressing a button
with their left index finger for the left option and with their right index finger
for the right option.
We therefore expect to see an LRP build up before either of the buttons is pressed.

First we will have to import the modules we need.
Make sure `mne_pipeline.py` is within the same directory of the script or
otherwise provide the relative path during the import.

In [3]:
# TODO: Check which imports are really needed
import os
import mne
import pandas as pd
import numpy as np
import mne_pipeline

We now import the data of a participant.
In a proper analysis script this might be done in a loop over all participants so that you can
deal with them one by one.

In [ ]:
triggers = {'Masked': 2, 'Reveal': 4, 'Left_choice': 8, 'Right_choice': 16, 'No_choice': 32}

eeg_prep = mne_pipeline.EEGPrep(
    eeg_path=os.path.join('Data', 'raw', 'kevin_raw.fif'),
    trigger_dict=triggers,
    participant_identifier='test_participant')

As you can see, the object `eeg_prep` is an instance of the EEGPrep class.
To initialize it we give it the path to our EEG file (which can be either in .bdf or .fif format),
the trigger dictionary and an identifier for the participant (which can also be a number).
The trigger dictionary contains the labels of the triggers as keys and the corresponding
value of the trigger channel as values. In this case `'masked'` means that at that time
the stimulus was presented but the numbers were not visible yet.
At this time the trigger channel (called `Status` or `Stim`) takes on the value of 2.

Now we have our `eeg_prep` object which has a bunch of attributes
(for example the raw data itself or the `participant_id` we just gave it) and methods.
The methods can either have a return value so that it will be used as
`new_value = eeg_prep.some_method()` or it can operate on the attributes within the object
 (again, for example the raw data) directly.

 The next two methods we are going to apply do exactly that.
 The `fix_channels` method removes the superfluous `1-` at the start of the channel names
 and sets the montage, which is what tells `mne` where on the head each electrode was attached.
 Then `set_references` automatically rereferences your signal to the average of the
 mastoid reference electrodes (the ones behind the ear) and creates new channels
 from the bipolar electrodes like ones capturing eye and hand movement.

In [ ]:
eeg_prep.fix_channels(n_ext_channels=9,
                      ext_ch_mapping=None)

eeg_prep.set_references(bipolar_dict=dict(eye_horizontal=['PO10', 'FT10'],
                                          eye_vertical=['HeRe', 'FT10'],
                                          right_hand=['HeLi', 'VeDo'],
                                          left_hand=['VeUp', 'EMG1a']))

As you can see, `fix_channels` also gives you the option to set the number of extra channels
(i.e. everything that is not included in the 64 electrode setup) and lets you provide your own
type mapping for those extra channels if desired (see the documentation in the code for details).

`set_references` takes a dictionary for the bipolar electrodes called `bipolar_dict`.
Here the keys will be used as the channel names for the newly generated channels and the
values should be a list containing the anode and cathode of the bipolar channel
(e.g. the electrodes above and below the eye for the `eye_vertical` channel).
If you use a reference channel other than the average of `PO9` and `FT9`, you can also
ise the `ref_ch` argument (not used here) to provide the channels to use as reference as a list.













